In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = 24016012 # 店舗ID
SPREADSHEET_KEY = '1RbIziGoSZbik5ekOYVv1oHtDNtNAizFXV4_inuuH5m8' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 469 #469 スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/18', '025870', '1', '1538.0', '-1203.0', '1', '1']
['08/18', '025870', '2', '2821.0', '2408.0', '3', '1']
['08/18', '025825', '3', '10513.0', '-8124.0', '1', '1']
['08/18', '025825', '4', '0', '0', '0', '0']
['08/18', '025665', '5', '15641.0', '-4212.0', '7', '2']
['08/18', '025665', '6', '7949.0', '13543.0', '14', '2']
['08/18', '025665', '7', '9744.0', '-4212.0', '3', '1']
['08/18', '025665', '8', '8718.0', '-7823.0', '0', '0']
['08/18', '025665', '9', '28718.0', '21066.0', '29', '3']
['08/18', '025665', '10', '513.0', '-902.0', '0', '0']
['08/18', '025665', '11', '20513.0', '5418.0', '16', '3']
['08/18', '025665', '12', '1282.0', '-1203.0', '0', '0']
['08/18', '025665', '13', '8462.0', '-7221.0', '0', '0']
['08/18', '025665', '14', '17436.0', '-12036.0', '2', '1']
['08/18', '025665', '15', '8718.0', '2709.0', '7', '1']
['08/18', '025665', '16', '7949.0', '-6920.0', '0', '0']
['08/18', '025665', '17', '13333.0', '-3610.0', '6', '2']
['08/18', '025665', '18', '19487.0', '-16249.0

['08/18', '025391', '150', '5641.0', '-2406.0', '4', '4']
['08/18', '025187', '151', '15641.0', '-4212.0', '9', '7']
['08/18', '025324', '152', '14359.0', '-2406.0', '13', '4']
['08/18', '025375', '153', '4359.0', '3311.0', '16', '1']
['08/18', '025325', '154', '15128.0', '-4513.0', '11', '6']
['08/18', '025431', '155', '15641.0', '1205.0', '19', '11']
['08/18', '025697', '156', '4359.0', '-3309.0', '2', '2']
['08/18', '025352', '157', '5128.0', '-1805.0', '4', '1']
['08/18', '025647', '158', '17436.0', '-2707.0', '28', '28']
['08/18', '025533', '159', '28974.0', '-9027.0', '26', '9']
['08/18', '025601', '160', '40256.0', '-902.0', '44', '44']
['08/18', '025523', '161', '23077.0', '-902.0', '28', '28']
['08/18', '025626', '162', '32051.0', '1506.0', '53', '51']
['08/18', '025777', '163', '16667.0', '-7823.0', '9', '8']
['08/18', '025747', '164', '15641.0', '-8425.0', '7', '6']
['08/18', '025760', '165', '7949.0', '1205.0', '32', '2']
['08/18', '025693', '166', '14872.0', '-6619.0', '15

['08/18', '025856', '289', '11282.0', '-5416.0', '3', '1']
['08/18', '025856', '290', '2821.0', '-2707.0', '0', '0']
['08/18', '025856', '291', '6410.0', '-4212.0', '1', '1']
['08/18', '025856', '292', '1026.0', '-1203.0', '0', '0']
['08/18', '025856', '293', '3077.0', '-3008.0', '0', '0']
['08/18', '025856', '294', '256.0', '-902.0', '0', '0']
['08/18', '025856', '295', '2308.0', '19561.0', '15', '1']
['08/18', '025856', '296', '18205.0', '-5416.0', '7', '2']
['08/18', '025839', '297', '0', '0', '0', '0']
['08/18', '025839', '298', '2821.0', '-2406.0', '0', '0']
['08/18', '025839', '299', '0', '0', '0', '0']
['08/18', '025764', '300', '0', '0', '0', '0']
['08/18', '025764', '301', '3590.0', '-1805.0', '1', '1']
['08/18', '025764', '302', '6923.0', '-4814.0', '1', '1']
['08/18', '025687', '303', '10513.0', '20765.0', '24', '24']
['08/18', '025687', '304', '14103.0', '-9629.0', '2', '2']
['08/18', '025687', '305', '11538.0', '-5115.0', '4', '4']
['08/18', '025687', '306', '11026.0', '-9

['08/18', '025667', '438', '8462.0', '-5717.0', '2', '2']
['08/18', '025667', '439', '15897.0', '302.0', '10', '4']
['08/18', '025295', '440', '3333.0', '-2406.0', '0', '0']
['08/18', '025549', '441', '23590.0', '2107.0', '19', '3']
['08/18', '025746', '442', '2821.0', '3612.0', '11', '1']
['08/18', '025700', '443', '9487.0', '9931.0', '34', '5']
['08/18', '025646', '444', '2308.0', '-1805.0', '0', '0']
['08/18', '025775', '445', '7436.0', '-5115.0', '1', '1']
['08/18', '025745', '446', '14872.0', '-8425.0', '9', '7']
['08/18', '025304', '447', '10513.0', '-2406.0', '5', '3']
['08/18', '025304', '448', '2564.0', '3913.0', '7', '1']
['08/18', '025668', '449', '10769.0', '-4513.0', '7', '2']
['08/18', '025668', '450', '6923.0', '-5115.0', '1', '1']
['08/18', '025671', '451', '4872.0', '-1805.0', '2', '1']
['08/18', '025788', '452', '5385.0', '1506.0', '5', '5']
['08/18', '025789', '453', '1026.0', '-1203.0', '0', '0']
['08/18', '025789', '454', '10513.0', '-9027.0', '1', '1']
['08/18', '

['08/18', '120140', '574', '875', '-972.0', '0', '0', '11']
['08/18', '120140', '575', '46', '0', '0', '0', '--']
['08/18', '120140', '576', '1445', '-1806.0', '0', '0', '11']
['08/18', '120150', '577', '5012', '-2824.0', '0', '12', '36']
['08/18', '120129', '578', '590', '139.0', '0', '2', '3']
['08/18', '120129', '579', '612', '-648.0', '0', '4', '1']
['08/18', '120129', '580', '627', '-370.0', '0', '3', '2']
['08/18', '120138', '581', '905', '-139.0', '0', '1', '4']
['08/18', '120138', '582', '1452', '1296.0', '0', '2', '5']
['08/18', '120138', '583', '1565', '-1343.0', '0', '2', '2']
['08/18', '120128', '584', '4099', '4352.0', '0', '1', '19']
['08/18', '120102', '585', '259', '93.0', '0', '1', '4']
['08/18', '120104', '586', '97', '-93.0', '0', '0', '--']
['08/18', '120104', '587', '199', '-185.0', '0', '0', '--']
['08/18', '120093', '588', '1970', '-972.0', '10', '0', '10']
['08/18', '120093', '589', '1969', '2824.0', '26', '0', '26']
['08/18', '120093', '590', '273', '185.0', '3

{'spreadsheetId': '1RbIziGoSZbik5ekOYVv1oHtDNtNAizFXV4_inuuH5m8',
 'updates': {'spreadsheetId': '1RbIziGoSZbik5ekOYVv1oHtDNtNAizFXV4_inuuH5m8',
  'updatedRange': 'S20RAW!F32375:M32598',
  'updatedRows': 224,
  'updatedColumns': 8,
  'updatedCells': 1792}}